In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import xgboost as xgb
from sklearn import preprocessing
from multiprocessing import Process
import multiprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pickle

In [2]:
df = pd.read_csv(r"C:\Users\Downloads\Intrusion Detection Dataset\Wednesday-WorkingHours.pcap_ISCX.csv")

In [4]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [26]:
labels = list(df[" Label"].unique())
print(labels)

[0, 4, 3, 2, 1, 5]


In [5]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,389,15206,17,12,3452,6660,1313,0,203.058823,425.778474,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
df.isnull().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692703 entries, 0 to 692702
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             692703 non-null  int64  
 1    Flow Duration                692703 non-null  int64  
 2    Total Fwd Packets            692703 non-null  int64  
 3    Total Backward Packets       692703 non-null  int64  
 4   Total Length of Fwd Packets   692703 non-null  int64  
 5    Total Length of Bwd Packets  692703 non-null  int64  
 6    Fwd Packet Length Max        692703 non-null  int64  
 7    Fwd Packet Length Min        692703 non-null  int64  
 8    Fwd Packet Length Mean       692703 non-null  float64
 9    Fwd Packet Length Std        692703 non-null  float64
 10  Bwd Packet Length Max         692703 non-null  int64  
 11   Bwd Packet Length Min        692703 non-null  int64  
 12   Bwd Packet Length Mean       692703 non-nul

In [8]:
df.shape

(692703, 79)

<b>

- There are no categorical features
- Each feature is numerical

In [9]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,389,15206,17,12,3452,6660,1313,0,203.058823,425.778474,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,53,32215,4,2,112,152,28,28,28.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
692699,53,324,2,2,84,362,42,42,42.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
692700,58030,82,2,1,31,6,31,0,15.500000,21.920310,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
692701,53,1048635,6,2,192,256,32,32,32.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


<h3> Transform actual labels into numerical data </h3>

In [10]:
from sklearn.preprocessing import LabelEncoder

In [28]:
labels = encoder.classes_
print(labels)

['BENIGN' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest' 'DoS slowloris'
 'Heartbleed']


In [11]:
encoder = LabelEncoder()
df[' Label']=encoder.fit_transform(df[' Label'])

In [12]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
3,389,15206,17,12,3452,6660,1313,0,203.058823,425.778474,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0


In [13]:
df[' Label'].value_counts()

 Label
0    440031
2    231073
1     10293
4      5796
3      5499
5        11
Name: count, dtype: int64

# here 0 = benign ,1 = dos golden eye,2= dos hulk ,3= dos slowhttptest 

In [14]:
df.isnull().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64

<h4> Data Clipping </h4>

In [15]:
Target= ' Label'
X = df.loc[:, df.columns != Target]
y = df.loc[:,Target]


lower_bound=0
upper_bound=150
X_clipped = np.clip(X, lower_bound, upper_bound) 

In [16]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_clipped)

- We want to select top 10 features that are useful in predicting the target variable

In [17]:
selector = SelectKBest(score_func=chi2, k=10)
selector.fit(X_imputed, y)
selected_features = selector.get_support(indices=True)
X_new = X_imputed[:, selected_features]
X_new_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.25)

In [18]:
print(selected_features)

[ 9 11 13 27 70 72 73 74 76 77]


In [19]:
X_new

array([[  0.        ,   6.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [ 31.4492376 ,   0.        ,  89.27877687, ...,   0.        ,
          0.        ,   0.        ],
       [150.        ,   0.        , 150.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ 21.92031022,   6.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        , 128.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        , 113.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [20]:
y.unique()

array([0, 4, 3, 2, 1, 5])

In [21]:
def plot_confusion_matrix(test_y, predict_y):
    C = confusion_matrix(test_y, predict_y)
    print("Number of misclassified points ",(len(test_y)-np.trace(C))/len(test_y)*100)
    # C = 6,6 matrix, each cell (i,j) represents number of points of class i are predicted class j
    
    A =(((C.T)/(C.sum(axis=1))).T)
    #divid each element of the confusion matrix with the sum of elements in that column
    
    # C = [[1, 2],
    #     [3, 4]]
    # C.T = [[1, 3],
    #        [2, 4]]
    # C.sum(axis = 1)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =1) = [[3, 7]]
    # ((C.T)/(C.sum(axis=1))) = [[1/3, 3/7]
    #                           [2/3, 4/7]]

    # ((C.T)/(C.sum(axis=1))).T = [[1/3, 2/3]
    #                           [3/7, 4/7]]
    # sum of row elements = 1
    
    B =(C/C.sum(axis=0))
    #divid each element of the confusion matrix with the sum of elements in that row
    # C = [[1, 2],
    #     [3, 4]]
    # C.sum(axis = 0)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =0) = [[4, 6]]
    # (C/C.sum(axis=0)) = [[1/4, 2/6],
    #                      [3/4, 4/6]] 
    
    labels = [1,2,3,4,5,6]
    cmap=sns.light_palette("green")
    # representing A in heatmap format
    print("-"*50, "Confusion matrix", "-"*50)
    plt.figure(figsize=(10,5))
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()

    print("-"*50, "Precision matrix", "-"*50)
    plt.figure(figsize=(10,5))
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    print("Sum of columns in precision matrix",B.sum(axis=0))
    
    # representing B in heatmap format
    print("-"*50, "Recall matrix"    , "-"*50)
    plt.figure(figsize=(10,5))
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    print("Sum of rows in precision matrix",A.sum(axis=1))

- How do we know if our model is going in the right direction?
- We can have a model that assigns classes on a random (Basically no training is happening)
- So any trained model should be way better than the random model
- This is a multiclass setting and so we would be using log loss

<h2> Random Model Benchmark </h2>

In [ ]:
# we need to generate 9 numbers and the sum of numbers should be 1
# one solution is to genarate 9 numbers and divide each of the numbers by their sum
# ref: https://stackoverflow.com/a/18662466/4084039

test_data_len = X_test.shape[0]


# Test-Set error.
#we create a output array that has exactly same as the test data
test_predicted_y = np.zeros((test_data_len,6))
for i in range(test_data_len):
    rand_probs = np.random.rand(1,6)
    test_predicted_y[i] = ((rand_probs/sum(sum(rand_probs)))[0])
print("Log loss on Test Data using Random Model",log_loss(y_test,test_predicted_y, eps=1e-15))

predicted_y =np.argmax(test_predicted_y, axis=1)
plot_confusion_matrix(y_test, predicted_y+1)

<h2> Random Forest Classifier </h2>

In [ ]:
# --------------------------------
# default parameters 
# sklearn.ensemble.RandomForestClassifier(n_estimators=10, criterion=’gini’, max_depth=None, min_samples_split=2, 
# min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
# min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, 
# class_weight=None)

# Some of methods of RandomForestClassifier()
# fit(X, y, [sample_weight])	Fit the SVM model according to the given training data.
# predict(X)	Perform classification on samples in X.
# predict_proba (X)	Perform classification on samples in X.

# some of attributes of  RandomForestClassifier()
# feature_importances_ : array of shape = [n_features]
# The feature importances (the higher, the more important the feature).


alpha=[15,20,25,30,45]
test_log_error_array=[]
for i in alpha:
    r_cfl=RandomForestClassifier(n_estimators=i,random_state=42,n_jobs=-1, class_weight='balanced')
    r_cfl.fit(X_new_train,y_train)
    sig_clf = CalibratedClassifierCV(r_cfl, method="sigmoid")
    sig_clf.fit(X_new_train, y_train)
    predict_y = sig_clf.predict_proba(X_test)
    test_log_error_array.append(log_loss(y_test, predict_y, labels=r_cfl.classes_, eps=1e-15))

for i in range(len(test_log_error_array)):
    print ('log_loss for c = ',alpha[i],'is',test_log_error_array[i])


best_alpha = np.argmin(test_log_error_array)

fig, ax = plt.subplots()
ax.plot(alpha, test_log_error_array,c='g')
for i, txt in enumerate(np.round(test_log_error_array,3)):
    ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],test_log_error_array[i]))
plt.grid()
plt.title("Cross Validation Error for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Error measure")
plt.show()

r_cfl=RandomForestClassifier(n_estimators=alpha[best_alpha],random_state=42,n_jobs=-1, class_weight='balanced')
r_cfl.fit(X_new_train,y_train)
sig_clf = CalibratedClassifierCV(r_cfl, method="sigmoid")
sig_clf.fit(X_new_train, y_train)
predict_y = sig_clf.predict_proba(X_new_train)
print ('log loss for train data',(log_loss(y_train, predict_y, labels=sig_clf.classes_, eps=1e-15)))
predict_y = sig_clf.predict_proba(X_test)
print ('log loss for test data',(log_loss(y_test, predict_y, labels=sig_clf.classes_, eps=1e-15)))
plot_confusion_matrix(y_test,sig_clf.predict(X_test))

In [24]:
from sklearn.metrics import accuracy_score
y_test_pred = sig_clf.predict(X_test)
print(accuracy_score(y_test, y_test_pred))

0.8177518824779415


<b>
    
- So far with just one parameter we can see good performance in our RandomForest Model and the test log loss seems to be < 0.35
- We can use other parameters apart from n_estimators and see if the RandomForestClassifier improves



In [29]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_test_pred, target_names=labels))

In [36]:
pickle.dump(sig_clf, open(r'C:\Users\Downloads\ids_rf_model.pkl', 'wb'))

<h4> Decision Trees </h4>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# --------------------------------
# default parameters 
# sklearn.ensemble.RandomForestClassifier(n_estimators=10, criterion=’gini’, max_depth=None, min_samples_split=2, 
# min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
# min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, 
# class_weight=None)

# Some of methods of RandomForestClassifier()
# fit(X, y, [sample_weight])	Fit the SVM model according to the given training data.
# predict(X)	Perform classification on samples in X.
# predict_proba (X)	Perform classification on samples in X.

# some of attributes of  RandomForestClassifier()
# feature_importances_ : array of shape = [n_features]
# The feature importances (the higher, the more important the feature).


alpha=[2, 3, 4, 5, 7, 8 ,9]
test_log_error_array=[]
for i in alpha:
    r_dt=DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=i, min_samples_split=10, min_samples_leaf = 20, class_weight='balanced')
    r_dt.fit(X_new_train,y_train)
    sig_dt = CalibratedClassifierCV(r_dt, method="sigmoid")
    sig_dt.fit(X_new_train, y_train)
    predict_y = sig_dt.predict_proba(X_test)
    test_log_error_array.append(log_loss(y_test, predict_y, labels=r_cfl.classes_, eps=1e-15))

for i in range(len(test_log_error_array)):
    print ('log_loss for c = ',alpha[i],'is',test_log_error_array[i])


best_alpha = np.argmin(test_log_error_array)

fig, ax = plt.subplots()
ax.plot(alpha, test_log_error_array,c='g')
for i, txt in enumerate(np.round(test_log_error_array,3)):
    ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],test_log_error_array[i]))
plt.grid()
plt.title("Cross Validation Error for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Error measure")
plt.show()

r_dt==DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=alpha[best_alpha], min_samples_split=10, min_samples_leaf = 20, class_weight='balanced')
r_dt.fit(X_new_train,y_train)
sig_dt = CalibratedClassifierCV(r_dt, method="sigmoid")
sig_dt.fit(X_new_train, y_train)
predict_y = sig_dt.predict_proba(X_new_train)
print ('log loss for train data',(log_loss(y_train, predict_y, labels=sig_dt.classes_, eps=1e-15)))
predict_y = sig_dt.predict_proba(X_test)
print ('log loss for test data',(log_loss(y_test, predict_y, labels=sig_dt.classes_, eps=1e-15)))
plot_confusion_matrix(y_test,sig_dt.predict(X_test))

In [32]:
from sklearn.metrics import accuracy_score
y_test_pred = sig_dt.predict(X_test)
print(accuracy_score(y_test, y_test_pred))

0.7940072527370998


In [ ]:
print(classification_report(y_test, y_test_pred, target_names=labels))

In [38]:
pickle.dump(sig_dt, open(r'C:\Users\Downloads\ids_dt_model.pkl', 'wb'))

<h3> XGBOOST Classifier </h3>

In [53]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
sample_weights = np.array([class_weights[label] for label in y_train])
print(sample_weights)

[0.26223321 0.26223321 0.49997017 ... 0.26223321 0.49997017 0.26223321]


In [ ]:
from sklearn.tree import DecisionTreeClassifier
# --------------------------------
# default parameters 
# sklearn.ensemble.RandomForestClassifier(n_estimators=10, criterion=’gini’, max_depth=None, min_samples_split=2, 
# min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
# min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, 
# class_weight=None)

# Some of methods of RandomForestClassifier()
# fit(X, y, [sample_weight])	Fit the SVM model according to the given training data.
# predict(X)	Perform classification on samples in X.
# predict_proba (X)	Perform classification on samples in X.

# some of attributes of  RandomForestClassifier()
# feature_importances_ : array of shape = [n_features]
# The feature importances (the higher, the more important the feature).


alpha=[2, 3, 4, 5, 7, 8 ,9]
test_log_error_array=[]
for i in alpha:
    xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y_train.value_counts()), use_label_encoder=False, eval_metric='mlogloss')

    xgb_model.fit(X_new_train,y_train)
    sig_xgb = CalibratedClassifierCV(xgb_model, method="sigmoid")
    sig_xgb.fit(X_new_train, y_train)
    predict_y = sig_xgb.predict_proba(X_test)
    test_log_error_array.append(log_loss(y_test, predict_y, labels=r_cfl.classes_, eps=1e-15))

for i in range(len(test_log_error_array)):
    print ('log_loss for c = ',alpha[i],'is',test_log_error_array[i])


best_alpha = np.argmin(test_log_error_array)

fig, ax = plt.subplots()
ax.plot(alpha, test_log_error_array,c='g')
for i, txt in enumerate(np.round(test_log_error_array,3)):
    ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],test_log_error_array[i]))
plt.grid()
plt.title("Cross Validation Error for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Error measure")
plt.show()

xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y_train.value_counts()), use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_new_train,y_train)
sig_xgb = CalibratedClassifierCV(r_dt, method="sigmoid")
sig_xgb.fit(X_new_train, y_train)
predict_y = sig_xgb.predict_proba(X_new_train)
print ('log loss for train data',(log_loss(y_train, predict_y, labels=sig_dt.classes_, eps=1e-15)))
predict_y = sig_dt.predict_proba(X_test)
print ('log loss for test data',(log_loss(y_test, predict_y, labels=sig_dt.classes_, eps=1e-15)))
plot_confusion_matrix(y_test,sig_xgb.predict(X_test))

In [41]:
from sklearn.metrics import accuracy_score
y_test_pred = sig_xgb.predict(X_test)
print(accuracy_score(y_test, y_test_pred))

0.7940072527370998


In [ ]:
print(classification_report(y_test, y_test_pred, target_names=labels))

In [43]:
pickle.dump(sig_dt, open(r'C:\Users\Downloads\ids_xgb_model.pkl', 'wb'))